In [ ]:
# External
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
# Local
from CGAN import CGanNet

NET_NAME = "0_Basic"
BATCH_SIZE = 32
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
LEARNING_RATE_DISC: float = 0.0002
LEARNING_RATE_GAN: float = 0.0002
DROPOUT_RATE: float = 0.4
DENSE_UNITS=128
CONV_LAYERS = 2
BATCH_NORM = False

In [ ]:
# Pobranie zbioru obrazów
(training_set, validation_set), dataset_info = tfds.load(
    'oxford_flowers102',
    split=['test', 'validation'],
    with_info=True,
    as_supervised=True,
)
# Rozmiar zbioru danych treningowych
dataset_size = dataset_info.splits['test'].num_examples

# Liczba klas obrazów
num_classes = dataset_info.features['label'].num_classes

# Przekształcenie obrazów do odpowiedniej postaci
def format_image(image, label):
    image = tf.cast(image, tf.float32)
    image = (image - 127.5) / 127.5
    image = tf.image.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))

    return image, label
train_batches = training_set.shuffle(dataset_size//4).map(format_image)

# Utworzenie słownika z kluczami jako numer etykiety i wartościami jako nazwa etykiety
get_label_name = dataset_info.features['label'].int2str
labels_strings = { _: get_label_name(_) for _ in range(num_classes) }

# Macierze numpy z obrazami i etykietami
numpy_train_batches_images = np.array([_[0] for _ in train_batches])
numpy_train_batches_labels = np.array([_[1] for _ in train_batches])

In [ ]:
cgan = CGanNet(net_name=NET_NAME,
               batch_size=BATCH_SIZE,
               image_width=IMAGE_WIDTH,
               image_height=IMAGE_HEIGHT,
               learning_rate_disc=LEARNING_RATE_DISC,
               learning_rate_gan=LEARNING_RATE_GAN,
               dropout_rate= DROPOUT_RATE,
               generator_dense_units=DENSE_UNITS,
               num_conv_layers=CONV_LAYERS,
               batch_norm=BATCH_NORM,
               number_of_channels=3,
               latent_dimension=100,
               training_data=numpy_train_batches_images,
               labels_data=numpy_train_batches_labels,
               number_of_classes=num_classes,
               labels_names = labels_strings)

# Zdefiniuj modele składające się na siec GAN
cgan.define_discriminator()
cgan.define_generator()
cgan.define_gan()
cgan.load_model()

In [ ]:
# Show images
cgan.show_sample_images_with_labels()

In [ ]:
cgan.show_one_image_with_label(label_num=20)